# Training Plots

In [1]:
# importing necessary libraries and the cnn architecture I defined

from cnn_architecture import CNN2Model
from utils import *
from load_datasets import load_and_prep_dataset

import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.io import loadmat
import copy
import pandas as pd
import seaborn as sns

2024-07-12 09:37:43.342508: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/net/projects/scratch/summer/valid_until_31_January_2025/epetersen/miniconda3/envs/thesis_gpu/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Train Loop and IMP loop

In [2]:
# modified train loop to also work with sparse networks (such that pruned weights remain frozen at 0.0)

def train_loop_sparse(train, test, model, num_epochs=5):
    
    # hyperparameters
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)
    loss_function= tf.keras.losses.CategoricalCrossentropy()
    
    # initializing training statistics
    train_accuracy = tf.keras.metrics.Accuracy(name='test_accuracy')
    test_accuracy = tf.keras.metrics.Accuracy(name='train_accuracy')
    train_losses = tf.keras.metrics.CategoricalCrossentropy(name='train_losses')
    test_losses = tf.keras.metrics.CategoricalCrossentropy(name='test_losses')
    train_acc = []
    test_acc = []
    train_l =[]
    test_l = []

    for epoch in tqdm(range(num_epochs), leave=False, desc="training epochs"):
        
        #train step
        for x, t in train:
            with tf.GradientTape() as tape:
                pred = model(x)
                loss = loss_function(t, pred)
                train_losses.update_state(t, pred)
                train_accuracy.update_state(tf.argmax(t,1), tf.argmax(pred,1))
            gradients = tape.gradient(loss, model.trainable_variables)
            
            # set gradients to zero for all pruned weights
            new_gradients = []
            for gradient_matrix, variables in zip(gradients, model.trainable_variables):
                gradient_matrix = tf.where(variables == 0.0, 0.0, gradient_matrix)
                new_gradients.append(gradient_matrix)
            optimizer.apply_gradients(zip(new_gradients, model.trainable_variables))
            
        # test step
        for x, t in test:
            pred = model(x)
            test_accuracy.update_state(tf.argmax(t,1), tf.argmax(pred,1))
            test_losses.update_state(t, pred)
        
        # updataing training statistics
        train_acc.append(train_accuracy.result().numpy())
        test_acc.append(test_accuracy.result().numpy())
        train_l.append(train_losses.result().numpy())
        test_l.append(test_losses.result().numpy())
        train_accuracy.reset_state()
        test_accuracy.reset_state()
        train_losses.reset_state()
        test_losses.reset_state()
        
    # collecting losses in a dictionary
    losses = {"test loss":test_l , "training loss":train_l , "test accuracy":test_acc , "training accuracy":train_acc}
    losses = pd.DataFrame.from_dict(losses, orient='columns')
    
    return  losses

## Training

- Winning Tickets
- Random Tickets
- Control Models

In [3]:
def train_control(train_dataset, test_dataset, nr):

    all_losses = pd.DataFrame()

    for i in range(nr):

        control_model = CNN2Model()
        control_losses = train_loop_sparse(train_dataset, test_dataset, control_model)
        control_losses["model_nr"] = i
        control_losses["Sparsity"] = "original"
        control_losses["epochs"] = control_losses.index

        all_losses = pd.concat([all_losses, control_losses], axis=0)
    
    return all_losses
    


In [4]:
def train_wts(train_dataset, test_dataset, dataset, nrs):

    all_losses = pd.DataFrame()

    for i in nrs:

        wt_model = CNN2Model()
        wt_model(list(train_dataset)[0][0])
        wt_model.load_weights(f"1b WTs/h5_format/WT_{dataset}_IMP_{i}.h5")


        wt_losses = train_loop_sparse(train_dataset, test_dataset, wt_model)
        wt_losses["model_nr"] = i
        wt_losses["Sparsity"] = "WT"
        wt_losses["epochs"] = wt_losses.index

        all_losses = pd.concat([all_losses, wt_losses], axis=0)
    
    return all_losses

In [5]:
def train_random_sparse(train_dataset, test_dataset, nrs):

    all_losses = pd.DataFrame()

    for i in nrs:

        random_sparse_model = CNN2Model()
        random_sparse_model(list(train_dataset)[0][0])
        random_sparse_model.load_weights(f"1b WTs/h5_format/RSN_{i}.h5")


        random_sparse_losses = train_loop_sparse(train_dataset, test_dataset, random_sparse_model)
        random_sparse_losses["model_nr"] = i
        random_sparse_losses["Sparsity"] = "RSN"
        random_sparse_losses["epochs"] = random_sparse_losses.index

        all_losses = pd.concat([all_losses, random_sparse_losses], axis=0)
    
    return all_losses

In [6]:
# reload the rsn random and change the sparsity column and save
random_sparse_losses = pd.read_csv('6b Training statistics/CIFAR_RSN_training_stats.csv')
random_sparse_losses["Sparsity"] = "RSN"
random_sparse_losses.to_csv('6b Training statistics/CIFAR_RSN_training_stats.csv', index=False)

wt_losses = pd.read_csv('6b Training statistics/CIFAR_WT_training_stats.csv')
wt_losses["Sparsity"] = "WT"
wt_losses.to_csv(f'6b Training statistics/CIFAR_WT_training_stats.csv', index=False)


control_losses = pd.read_csv("6b Training statistics/CIFAR_original_training_stats.csv")

coll_stats = pd.concat([control_losses,wt_losses,random_sparse_losses], axis =0)
coll_stats.to_csv(f'6b Training statistics/CIFAR_collective_training_stats.csv', index=False)
# reload all three training statistics and 

In [7]:
def get_train_statistics(dataset):
    
    nr = 15

    train_dataset, test_dataset = load_and_prep_dataset(dataset, batch_size=60, shuffle_size=512)

    control_stats = train_control(train_dataset, test_dataset, nr)
    control_stats["dataset"]=dataset
    #display(control_stats)
    control_stats.to_csv(f'6b Training statistics/{dataset}_original_training_stats.csv', index=False)

    wts_stats = train_wts(train_dataset, test_dataset, dataset, range(nr))
    wts_stats["dataset"]=dataset
    #display(wts_stats)
    wts_stats.to_csv(f'6b Training statistics/{dataset}_WT_training_stats.csv', index=False)

    random_stats = train_random_sparse(train_dataset, test_dataset, range(nr))
    random_stats["dataset"]=dataset
    #display(random_stats)
    random_stats.to_csv(f'6b Training statistics/{dataset}_RSN_training_stats.csv', index=False)

    coll_stats = pd.concat([control_stats,wts_stats,random_stats], axis =0)
    coll_stats.to_csv(f'6b Training statistics/{dataset}_collective_training_stats.csv', index=False)

In [8]:
#get_train_statistics("CIFAR")
get_train_statistics("CINIC")
get_train_statistics("SVHN")

Found 89326 files belonging to 10 classes.
Using 80394 files for training.


2024-07-12 09:37:54.597032: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 09:37:54.646783: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 09:37:54.646971: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Found 89326 files belonging to 10 classes.
Using 8932 files for validation.


AttributeError: module 'keras.api._tf_keras.keras.layers' has no attribute 'experimental'